In [ ]:
import pandas as pd
from textblob import TextBlob

# Load your CSV
df = pd.read_csv(r"C:\Users\linda\Desktop\labeled_textblob.csv")

# Combine subject and body
df['message'] = df['Subject'].fillna('') + " " + df['body'].fillna('')

# Define function to get sentiment
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return 'Positive'
    elif polarity < -0.1:
        return 'Negative'
    else:
        return 'Neutral'

# Apply to messages
df['sentiment'] = df['message'].apply(get_sentiment)

# Save results
df.to_csv('labeled_textblob.csv', index=False)
print(df[['Subject', 'sentiment']].head())
